#  HW1 NLP Anna Aksenova

В этом ноутбуке мы будем проводить анализ тональности на текстах из отзывов на IMDB о Гарри Поттере.

In [1]:
import re
from pprint import pprint
import time
import requests
from tqdm.auto import tqdm

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

Берем ссылку на IMDB

In [2]:
link = 'https://www.imdb.com/title/tt0241527/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0 '

In [3]:
result = requests.get(link)
html = result.text

In [4]:
# эта функция открывает IMDB и кликает на кнопку, открывающую продолжение комментариев
def get_comments(url_add, maxclicks):
    
    progress_bar = tqdm(total = maxclicks)

    source_code = requests.get(url_add)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')


    driver = webdriver.Chrome('/Users/annaaksenova/Desktop/chromedriver')
    wait = WebDriverWait(driver, 100)

    driver.get(url_add)
    
    clicks = 0
    while True:
        clicks += 1
        if clicks <= maxclicks:
            more_button = wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "ipl-load-more__button"))).click()


        else:
            break
        progress_bar.update(1)
        
    time.sleep(25)
    source_code = driver.page_source

    plain_text = source_code
    soup = BeautifulSoup(plain_text, 'html.parser')

    database = []
    pageno = 1
    rating_list = []
    review_list = []

    for r in soup.find_all('div', {'class': 'lister-item-content'}):

        if "ipl-ratings-bar" in str(r):

            for rating in r.find_all('span', {'class': 'rating-other-user-rating'}):
                rating = str(rating.text).strip()
                if rating == '':
                    rating = 'NaN'
                rating_list.append(rating)

        for review in r.find_all('div', {'class': 'text'}):
            review = str(review.text)
            if review == '':
                review = 'NaN'
                review_list.append(review)
            else:
                review_list.append(review)
        progress_bar.close()
    return rating_list, review_list

In [5]:
rating_list, review_list = get_comments(link, 30)

Выберем комментарии, в которых проставлены оценки

In [6]:
com_rate = [(int(com[0].split('/')[0]), com[1]) for com in zip(rating_list, review_list) if not com[0].isalpha()]

Будем считать все оценки от 8 и выше положительными

In [7]:
positive = [com for com in com_rate if com[0]>=8]

Все оценки от 3 и ниже будем считать отрицательными

In [8]:
negative = [com for com in com_rate if com[0]<=3]

Посмотрим, сколько у нас вообще отзывов каждого типа

In [9]:
len(positive)

137

In [10]:
len(negative)

54

Сбалансируем выборку

In [11]:
positive_val = [(1, i) for i in positive[len(negative) - 10:len(negative)]]
negative_val = [(0, i) for i in negative[len(negative) - 10:]]

## Обработка текста

In [12]:
from collections import Counter 

import nltk
from nltk.stem import WordNetLemmatizer

In [13]:
lemmatizer = WordNetLemmatizer()

In [14]:
# эта функция собирает частотный словарь в 100 слов по дефолту
def collect_freqlist(reviews, max_len=100):
    
    freqlist = Counter()
    for text in reviews:
        for word in nltk.word_tokenize(text[1].lower()):
            if word.isalpha():
                word = lemmatizer.lemmatize(word)
                freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

In [15]:
positive_list = collect_freqlist(positive[:len(negative) - 10])
negative_list = collect_freqlist(negative[:len(negative) - 10])

In [16]:
positive_list = set(positive_list.keys())
negative_list = set(negative_list.keys())
common = positive_list.intersection(negative_list)

In [17]:
positive_list.symmetric_difference_update(common)
negative_list.symmetric_difference_update(common)

## Добавляем биграммы

In [18]:
import spacy

nlp = spacy.load("en_core_web_sm")

Первый тип биграммы – не + прилагательное. Это поможет нам выделить негативные отзывы, тк там встречаются прилагательные с положительной оценкой под отрицанием, а униграммы это не выделяют.

In [57]:
def collect_neglist(reviews, max_len=20):
    
    freqlist = Counter()
    for text in reviews:
        no = ''
        doc = nlp(text[1].lower())
        for sent in doc.sents:
            for t in sent:
                if no == '':
                    if t.lemma_ == "not":
                        no = "not"
                else:
                    if t.pos_ == 'ADJ':
                        freqlist[no + ' ' + lemmatizer.lemmatize(t.text)] += 1 
                    no = ''


    return dict(freqlist.most_common(max_len))

In [58]:
positive_list.difference_update(common)
negative_list.difference_update(common)

In [59]:
positive_neglist = collect_neglist(positive[:len(negative) - 10])
negative_neglist = collect_neglist(negative[:len(negative) - 10])

In [60]:
positive_neglist = set(positive_neglist.keys())
negative_neglist = set(negative_neglist.keys())
common_neg = positive_neglist.intersection(negative_neglist)

In [61]:
positive_neglist.difference_update(common_neg)
negative_neglist.difference_update(common_neg)

Второй тип биграммы – прил + сущ. Так мы поймем, за что хвалят и за что ругают фильм. Например beloved character VS bad acting.

In [62]:
def collect_adjlist(reviews, max_len=20):
    
    freqlist = Counter()
    for text in reviews:
        adj = ''
        doc = nlp(text[1].lower())
        for sent in doc.sents:
            for t in sent:
                if adj == '':
                    if t.pos_ == "ADJ":
                        adj = lemmatizer.lemmatize(t.text)
                else:
                    if t.pos_ == 'NOUN':
                        freqlist[adj + ' ' + lemmatizer.lemmatize(t.text)] += 1 
                        adj = ''
                    elif t.pos_ != 'ADJ':
                        adj = ''  
                    else:
                        adj = t.text
    return dict(freqlist.most_common(max_len))

In [63]:
positive_adjlist = collect_adjlist(positive[:len(negative) - 10])
negative_adjlist = collect_adjlist(negative[:len(negative) - 10])

In [64]:
positive_adjlist = set(positive_adjlist.keys())
negative_adjlist = set(negative_adjlist.keys())
common_adj = positive_adjlist.intersection(negative_adjlist)

In [65]:
positive_adjlist.difference_update(common_adj)
negative_adjlist.difference_update(common_adj)

Третий тип групп – глагол + наречие. Таким образом мы найдем группы типа "played terribly" или "fitted perfectly"

In [66]:
def collect_advlist(reviews, max_len=20):
    
    freqlist = Counter()
    for text in reviews:
        adv = ''
        doc = nlp(text[1].lower())
        for sent in doc.sents:
            for t in sent:
                if adv == '':
                    if t.pos_ == "VERB":
                        adv = lemmatizer.lemmatize(t.text)
                else:
                    if t.pos_ == 'ADV':
                        freqlist[adv + ' ' + lemmatizer.lemmatize(t.text)] += 1 
                        adv = ''
                    elif t.pos_ != 'VERB':
                        adv = ''  
                    else:
                        adv = t.text
    return dict(freqlist.most_common(max_len))

In [67]:
positive_advlist = collect_advlist(positive[:len(negative) - 10])
negative_advlist = collect_advlist(negative[:len(negative) - 10])

In [68]:
positive_advlist = set(positive_advlist.keys())
negative_advlist = set(negative_advlist.keys())
common_adv = positive_advlist.intersection(negative_advlist)

In [69]:
positive_advlist.difference_update(common_adv)
negative_advlist.difference_update(common_adv)

Соберем множества положительного и отрицательного словаря

In [70]:
all_pos = positive_list.union(positive_adjlist, positive_neglist, positive_advlist)
all_neg = negative_list.union(negative_adjlist, negative_neglist, negative_advlist)

In [71]:
len(all_neg)

60

## Определение типа отзыва по словам

In [72]:
def rate_detect(text):
    
    words = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text[1][1].lower())]
    words = ' '.join(words)
    words = words.replace("""n't""", 'not')
    pos_rank = 0
    neg_rank = 0
    for pos, neg in zip(all_pos, all_neg):
        if pos in words:
            pos_rank += 1
        if neg in words:
            neg_rank += 1
    if pos_rank >= neg_rank:
        check = 1
    else:
        check = 0
    return check

In [73]:
answers = []
for text in positive_val + negative_val:
    answers.append(rate_detect(text))

In [74]:
from sklearn.metrics import accuracy_score

In [75]:
print("Доля верных ответов:", accuracy_score(answers, [1 if i < 10 else 0 for i in range(20)]))

Доля верных ответов: 0.6


На самом деле ничего не улучшилось. Это произошло потому что очень маленькие тренировочные и тестовые выборки